# Using label Propogation pipeline to label and classify mock data

This notebook demonstrate how to use the label propagation code to label clothing data. The data used in this example is a Mock dataset randomly generated using the 100  most common words for each category in the actual data used in the paper that accompanies this work. 



In [1]:
# Python imports
import json
import numpy as np
import pandas as pd
from importlib import reload

# plotting imports
from matplotlib import pylab as plt
import seaborn as sns
%matplotlib notebook

# scikit lean imports
from sklearn.model_selection import train_test_split, cross_val_score, GridSearchCV
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import confusion_matrix, classification_report

# NLP imports
from fuzzywuzzy import fuzz
from nltk.metrics import distance

# Custom imports of files to run label propagation pipeline
import labelling_pipeline
import Word_vectors

/home/edd/.local/share/virtualenvs/labelpropagation_clothing-66lVHivS/lib/python3.6/site-packages/fuzzywuzzy/fuzz.py:35: UserWarning: Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning
  warnings.warn('Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning')


In [2]:
import importlib

In [3]:
importlib.reload(labelling_pipeline)

<module 'labelling_pipeline' from '/home/edd/Documents/Projects/labelpropagation_clothing/labelling_pipeline.py'>

### Read in data
First read in the seed labels that will be used for fuzzy matching and label propagation

In [4]:
# seed labels
with open('seed_labels.json', 'r') as f:
    seed_labels = json.load(f)

Now read in the mock data to be labelled 

In [5]:
# Read in data
df_mock = pd.read_csv('clothing_mock_data.csv',
                      encoding="ISO-8859-1",
                      index_col=0)

Prevelance of items in the mock dataset

In [6]:
print('Number of items for each label')
for lab in np.sort(df_mock.true_label.unique()):
    print(
        f"{len(df_mock[df_mock['true_label'] == lab])} items with label {lab} "
    )

Number of items for each label
1104 items with label 0 
434 items with label 1 
393 items with label 2 
75 items with label 3 
576 items with label 4 
514 items with label 5 
395 items with label 6 
243 items with label 7 
261 items with label 8 


**create a test train split**

In [7]:
X_train, X_test, y_train, y_test = train_test_split(
    df_mock.drop('true_label', axis=1),
    df_mock['true_label'],
    test_size=0.20,
    stratify=df_mock['true_label'])

#### List of primary genders, categories 
This is used to help with selecting sub-set to match with and to clean the fuzzy matching to ensure maximum accuracy for the labels going into label propagation

In [8]:
# gender for each category
divisions = [
    'Womens', 'Men', 'Womens', 'Womens', 'Boys', 'Girls', 'Men', 'Men'
]
# broad categories for fuzzy matching cleaning
categories = [
    'Coats', 'Shirts', 'Fitness & activewear', 'Swim', 'Jeans', 'Tops',
    'Underwear', 'Underwear'
]

# list of stopwords for use in word vectors
stop = [
    'pack', 'print', 'printed', 'sleeve', 'long', 'fit', 'f', 't', 's', 'asos',
    'stripe', 'slim', 'running', 'active'
]

# names for output cleaned/labeled dataframes
outs = ['coats', 'shirts', 'shorts', 'swim', 'jeans', 'tops', 'pants', 'socks']

## Perform Label propogation for each category in turn 


These are the parameters used for fuzzy matching. 

Each matching method used requires a dictionary of the require parameters.

These are put into a list to allow parameters for different methods to be run in the pipeline.

The final fuzzy matched class for an is a modal (popular vote) decision from the result of all the methods used except in cases where one or more method shows a below the low threshold similarity (neg parameter), essentially a veto on the decision.



In [9]:
matching_list = [{
    "label": "label_fuz",
    "func": fuzz.partial_ratio,
    "thresh": 0.70,
    "neg": 0.25,
    "greater": True,
    "less": False
}, {
    "label": "label_edit",
    "func": distance.edit_distance,
    "thresh": 0.30,
    "neg": 0.75,
    "greater": False,
    "less": True
}, {
    "label": "label_jaccard",
    "func": distance.jaccard_distance,
    "thresh": 0.30,
    "neg": 0.75,
    "greater": False,
    "less": True
}]

word_vectorizer_params = {
    "epochs": 200,
    "print_sim": False,
    "ngram_range": (1, 2),
    "doc2vec_thresh": 0.5,
    "tfidf_thresh": 0.5,
    "count_thresh": 0.5,
    "fast_thresh": 0.5,
    "word2vec_thresh": 0.5,
    "doc2vec_params": {
        'alpha': 0.01,
        'kernel': 'knn',
        'n_neighbors': 6
    },
    "tfidf_params": {
        'alpha': 0.01,
        'kernel': 'knn',
        'n_neighbors': 6
    },
    "count_params": {
        'alpha': 0.01,
        'kernel': 'knn',
        'n_neighbors': 6
    },
    "fast_params": {
        'alpha': 0.01,
        'kernel': 'knn',
        'n_neighbors': 6
    },
    "word2vec_params": {
        'alpha': 0.01,
        'kernel': 'knn',
        'n_neighbors': 6
    }
}

In [10]:
# parameters for the label propagation for each kind of word vector in tern
param_doc2vec = {'alpha': 0.01, 'kernel': 'knn', 'n_neighbors': 6}
param_tfidf = {'alpha': 0.01, 'kernel': 'knn', 'n_neighbors': 6}
param_count = {'alpha': 0.01, 'kernel': 'knn', 'n_neighbors': 6}

# empty dictinary to collect results of each category
clean_frames = dict()

# loop through the labelling pipeline for each output category
for i in range(len(outs)):
    print('\n** ', outs[i])

    # select only the correct gender - assume the gender column is reliabel

    temp = X_train.loc[(X_train['division'] == divisions[i])]

    # perform matching and cleaning
    cleaned = labelling_pipeline.labelling_pipeline(
        data=temp,
        names=temp["name"],
        matching_params=matching_list,
        word_vectorizer_params=word_vectorizer_params,
        n_words=15,
        stop_words=stop,
        item_id=1,
        label_names=seed_labels[str(i)],
        common_words=True,
        cat_clean=True,
        division=divisions[i],
        category=categories[i],
        wv_names=("word2vec", "tfidf", "count"),
        verbose=True,
        use_difflib=True)
    
    clean_frames[outs[i]] = cleaned

print('labelled')


**  coats


/home/edd/.local/share/virtualenvs/labelpropagation_clothing-66lVHivS/lib/python3.6/site-packages/pandas/core/indexing.py:337: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self.obj[key] = _infer_fill_value(value)
/home/edd/.local/share/virtualenvs/labelpropagation_clothing-66lVHivS/lib/python3.6/site-packages/pandas/core/indexing.py:517: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self.obj[item] = s


/home/edd/.local/share/virtualenvs/labelpropagation_clothing-66lVHivS/lib/python3.6/site-packages/sklearn/semi_supervised/label_propagation.py:297: RuntimeWarning: invalid value encountered in true_divide
  self.label_distributions_ /= normalizer
/home/edd/.local/share/virtualenvs/labelpropagation_clothing-66lVHivS/lib/python3.6/site-packages/pandas/core/indexing.py:337: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self.obj[key] = _infer_fill_value(value)
/home/edd/.local/share/virtualenvs/labelpropagation_clothing-66lVHivS/lib/python3.6/site-packages/pandas/core/indexing.py:517: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation

IndexError: index 1 is out of bounds for axis 1 with size 1

### Combine the labels across iterations 

If a label is assigned in more than one loop it is assigned -1 for unlabelled
If a label is assigned and then a 0 is also assigned the label outranks the 0
If no label has been assigned ever then assume it is junk


In [ ]:
# combine_labels for all categories
label_dict={'other':0,'coats':1, 'shirts':2, 'shorts':3, 'swim':4, 'jeans':5, 'tops':6, 'pants':7, 'socks':8}

X_train['label_combined'] = -10000
X_train['lab_count'] =0 
X_train['zero_count'] = 0

# loop through the 
for i, frame in enumerate(list(clean_frames.keys())):
    X_train[label_dict[frame]]=np.nan
    index = clean_frames[frame].index
    
    X_train.loc[index, label_dict[frame]] = clean_frames[frame]['label_labprop_mode']
    X_train.loc[(X_train[label_dict[frame]]==1), 'lab_count']+=1
    X_train.loc[(X_train[label_dict[frame]]==0), 'zero_count']+=1
    
    X_train.loc[(X_train['lab_count']>1), 'label_combined']=-1
    
    X_train.loc[(X_train[label_dict[frame]]==1)&
               (X_train['lab_count']==1), 'label_combined'] = label_dict[frame]
    X_train.loc[(X_train[label_dict[frame]]==0)&
               (X_train['zero_count']==i+1), 'label_combined'] = 0
    X_train.loc[(X_train[label_dict[frame]]==-1)&
               (X_train['lab_count']==0), 'label_combined']=-1
    
    X_train.loc[X_train['label_combined']==-10000, 'label_combined']=0

## Examine the results of the label propagation pipeline

**Numer of labels assigned**

In [ ]:
print(f'Number of labels assigned: {len(X_train[X_train.label_combined>-1])}')
print(f'Number of unlabelled items: {len(X_train[X_train.label_combined==-1])}')

#### Classification report for labelled items

In [ ]:
print(classification_report(y_train.loc[(X_train.label_combined>-1)],
                            X_train.loc[(X_train.label_combined>-1),'label_combined']))

#### Confusion matrix for labelled items

In [ ]:
mat = confusion_matrix(y_train.loc[(X_train.label_combined>-1)],
                       X_train.loc[(X_train.label_combined>-1),'label_combined'])
fig,ax=plt.subplots(figsize=(6,6))
sns.heatmap(mat.T,square=True,annot=True,fmt='d',cbar=False)

ax.set_yticklabels(['Junk','W Coats','M Casual Shirt', 'W sports shorts','W swim',
               'B Jeans','G fashion top','M Underwear','M Socks'][::-1],rotation=45)

ax.set_xticklabels(['Junk','W Coats','M Casual Shirt', 'W sports shorts','W swim',
               'B Jeans','G fashion top','M Underwear','M Socks'],rotation=45)

ax.tick_params(axis='both', which='major', labelsize=8)
ax.set_xlabel('Actual')
ax.set_ylabel('Predicted')

ax.set_title('No Cleaning')
ax.set_xlabel('Actual')
ax.set_ylabel('Propogated')
fig.tight_layout()    

The labels have been propagated accuratly for most labels. This has not worked as well for the Mens Socks and Mens Pants categories. This is likely due to the shared language between socks and underwear as well as them falling in to the same broad category underwear. As this is a mock dataset the word order of the product names does not make sense and this may of impacted the quality of the fuzzy matching

# Train a classifiers

Now we have the labelled data we can use it to train a classifier. 
In this example notebook we have only used the fastText vectors in the classifier it is quite easy to instead use other word vectors.

### Create word vectors
Want to create the word vectors from the largest possible vocabulary so use the whole mock dataset

In [ ]:
# get words for word_vectors
names=Word_vectors.remove_stopwords(df_mock['name'])

print('create models')

print('tfidf')
tfidf_mat,tf_mod = Word_vectors.TF_IDF(names.values, ngram_range=(1, 2),
                                       print_sim=False,max_features=50)
df_mock['tfidf'] = tfidf_mat.toarray().tolist()

print('count_vec')
count_mat,count_mod = Word_vectors.count_vectorizer(names.values, ngram_range=(1, 2),
                                                    print_sim=False,max_features=50)
df_mock['count_vec'] = count_mat.toarray().tolist()
names
ft_vectors, model_fasttext = Word_vectors.fasttext_vectors(names)
df_mock['fasttext_vec'] = ft_vectors.tolist()

w2v_vectors, model_w2v = Word_vectors.fit_word2vec(names)
df_mock['w2v_vec'] = w2v_vectors.tolist()


**Infer word vectors for labelled training data and the test data**

In [ ]:
reload(Word_vectors)
print('infer labelled data vectors')

names_train=Word_vectors.remove_stopwords(X_train['name'])


X_train['tfidf'] = tf_mod.transform(names_train).toarray().tolist()
X_train['count_vec'] = count_mod.transform(names_train).toarray().tolist()
X_train['fasttex']=Word_vectors.fasttext_infvec(names_train,model_fasttext).tolist()
X_train['word2vec']=Word_vectors.word2vec(names_train,model_w2v).tolist()

print('infer unlabelled vectors')
names_test=Word_vectors.remove_stopwords(X_test['name'])
    
X_test['tfidf'] = tf_mod.transform(names_test).toarray().tolist()
X_test['count_vec']=count_mod.transform(names_test).toarray().tolist()
X_test['fasttex']=Word_vectors.fasttext_infvec(names_test,model_fasttext).tolist()
X_test['word2vec']=Word_vectors.word2vec(names_test,model_w2v).tolist()


### Create label encodings for the categorical columns division, category and and subcategory 
The encodings are created on the shole test data then applied to test and train

In [ ]:
div_code = LabelEncoder().fit(df_mock.division)
cat_code = LabelEncoder().fit(df_mock.category)
subcat_code = LabelEncoder().fit(df_mock.subcategory)

In [ ]:
X_train['division_code']=div_code.transform(X_train.division)
X_train['category_code']=cat_code.transform(X_train.category)
X_train['subcategory_code']=subcat_code.transform(X_train.subcategory)


X_test['division_code']=div_code.transform(X_test.division)
X_test['category_code']=cat_code.transform(X_test.category)
X_test['subcategory_code']=subcat_code.transform(X_test.subcategory)



### Create the training dataset with only the products assigned a label, not -1 prodcuts

In [ ]:
labelled=X_train[X_train['label_combined']>-1]
labelled_y=y_train[X_train['label_combined']>-1]

**Need to stack together all the inputs for the classifiers inorder to use the word vectors with the categorical variables**

In [ ]:
# Stack X train inputs
X_train_in = np.stack(labelled[['division_code', 'category_code', 'subcategory_code']].values)
X_train_in = np.concatenate((X_train_in,np.stack(labelled['fasttex'])),axis=1)


# # stack unlabelled inputs 
X_test_in = np.stack(X_test[['division_code', 'category_code', 'subcategory_code']].values)
X_test_in = np.concatenate((X_test_in,np.stack(X_test['fasttex'])),axis=1)


## Decision Tree

Build a decision tree with the fastText vectors. No rigourious hyper-parameter search has been undertaken only a basic grid search.

In [ ]:
param_grid = [{'max_depth':[5,10,20,30,50],'min_samples_leaf':[5,10,15,20,25],'min_samples_split':[2,5,7,10,20]}]
dt_clf = GridSearchCV(estimator= DecisionTreeClassifier(class_weight='balanced'),param_grid=param_grid,cv=5)
dt_clf.fit(X_train_in,labelled['label_combined'])

dt_params = dt_clf.best_params_

print(f'Best fit parameters: {dt_params}')

dt_clf = DecisionTreeClassifier(**dt_params)
dt_clf.fit(X_train_in,labelled['label_combined'])

ind = np.argsort(dt_clf.feature_importances_)[::-1]

print('\n*** Training labelled data set against true labels***')

print(classification_report(labelled_y,dt_clf.predict(X_train_in)))


predict_dt=dt_clf.predict(X_test_in)
print('\n*** Test split of labelled data set ***')
print(classification_report(y_test,predict_dt))

mat = confusion_matrix(y_test,predict_dt)
fig,ax=plt.subplots(figsize=(6,6))
sns.heatmap(mat.T,square=True,annot=True,fmt='d',cbar=False)
ax.set_yticklabels(['Junk','W Coats','M Casual Shirt', 'W sports shorts','W swim',
               'B Jeans','G fashion top','M Underwear','M Socks'][::-1],rotation=45)

ax.set_xticklabels(['Junk','W Coats','M Casual Shirt', 'W sports shorts','W swim',
               'B Jeans','G fashion top','M Underwear','M Socks'],rotation=45)

ax.tick_params(axis='both', which='major', labelsize=8)
ax.set_xlabel('Actual')
ax.set_ylabel('Predicted')
fig.tight_layout()


The decision tree works reasonaly well with the poorly performing items being the same as those that did not perform so well in propagation

## Random Forest

Build a random forest with the fastText vectors. No rigourious hyper-parameter search has been undertaken only a basic grid search.

In [ ]:
param_grid = [{'n_estimators':[5,10,20,30,50],'max_depth':[5,10,20,30,50,100],'min_samples_leaf':[1,3,5,10,15]}]
rf_clf = GridSearchCV(estimator= RandomForestClassifier(class_weight='balanced'),param_grid=param_grid,cv=5)
rf_clf.fit(X_train_in,labelled['label_combined'])
rf_parm = rf_clf.best_params_

print(f'Best fit parameters: {dt_params}')

rf_clf = RandomForestClassifier(**rf_parm,class_weight='balanced') #max_depth=10,min_samples_leaf=5)
rf_clf.fit(X_train_in,labelled['label_combined'])


print('\n*** Training labelled data set against true labels***')
print(classification_report(labelled_y,rf_clf.predict(X_train_in)))


predict_rf=rf_clf.predict(X_test_in)
print('\n*** Test split of labelled data set ***')
print(classification_report(y_test,predict_rf))

mat = confusion_matrix(y_test,predict_rf)
fig,ax=plt.subplots(figsize=(6,6))
sns.heatmap(mat.T,square=True,annot=True,fmt='d',cbar=False)
ax.tick_params(axis='both', which='major', labelsize=8)
ax.set_xlabel('Actual')
ax.set_ylabel('Predicted')

predict_rf=rf_clf.predict(X_test_in)

ax.set_yticklabels(['Junk','W Coats','M Casual Shirt', 'W sports shorts','W swim',
               'B Jeans','G fashion top','M Underwear','M Socks'][::-1],rotation=45)

ax.set_xticklabels(['Junk','W Coats','M Casual Shirt', 'W sports shorts','W swim',
               'B Jeans','G fashion top','M Underwear','M Socks'],rotation=45)

ax.tick_params(axis='both', which='major', labelsize=8)
ax.set_xlabel('Actual')
ax.set_ylabel('Predicted')
fig.tight_layout()
# fig.savefig('fasttext_rf_confusion.png')    

The random forest is performing better accross most categories but is unable to distinguish socks and underwear. Aain this is where the label propagation performed worse. 

## Suport vector machine

Build a SVM with the fastText vectors. No rigourious hyper-parameter search has been undertaken only a basic grid search.

In [ ]:
param_grid = [{'C':[10,100,1000,2000,5000,10000,20000],'gamma':[0.0,0.00001,0.00005,0.0001,0.01],'kernel':['rbf']}]

svm_clf = GridSearchCV(estimator=SVC(probability=True),param_grid=param_grid,cv=5)
svm_clf.fit(X_train_in,labelled['label_combined'])
svm_param = svm_clf.best_params_
print(f"Best fit parameters: {dt_params}")

svm_clf = SVC(**svm_param, probability=True) #max_depth=10,min_samples_leaf=5)
svm_clf.fit(X_train_in,labelled['label_combined'])

predict_svm=svm_clf.predict(X_test_in)

print('\n*** Training labelled data set against true labels***')
print(classification_report(labelled_y,svm_clf.predict(X_train_in)))


print('\n*** Test split of labelled data set ***')
print(classification_report(y_test,predict_svm))

mat = confusion_matrix(y_test,predict_svm)
fig,ax=plt.subplots(figsize=(6,6))
sns.heatmap(mat.T,square=True,annot=True,fmt='d',cbar=False)
ax.tick_params(axis='both', which='major', labelsize=8)
ax.set_xlabel('Actual')
ax.set_ylabel('Predicted')

predict_proba=svm_clf.predict_proba(X_test_in)


ax.set_yticklabels(['Junk','W Coats','M Casual Shirt', 'W sports shorts','W swim',
               'B Jeans','G fashion top','M Underwear','M Socks'][::-1],rotation=45)

ax.set_xticklabels(['Junk','W Coats','M Casual Shirt', 'W sports shorts','W swim',
               'B Jeans','G fashion top','M Underwear','M Socks'],rotation=45)

ax.tick_params(axis='both', which='major', labelsize=8)
ax.set_xlabel('Actual')
ax.set_ylabel('Predicted')
fig.tight_layout()

The SVM has given similar results to the decision tree, performing very well except where the label propagation has done poorly.


This notebook is intended to demonstate the methods of paper Semi-supervised machine learning with word embedding for classification in price statistics, Martindale et.al. using a mock dataset. 
We are not attempting to recreate the results of the paper as we can not make the data publicly avalible instead the mock dataset gives a demonstataion of the code. We do however see generally similar trends as with the true data. For more methods data see the paper.
